In [1]:
import numpy as np
import pandas as pd
import copy
import queue
dx=[-1, 0, 1, 0]
dy=[0, 1, 0, -1]

In [2]:
def state2dict(state):
    pos_dict = {}
    for i in range(3):
        for j in range(3):
            pos_dict[i*3 + j] = state[i][j]
    return pos_dict

In [3]:
def generate_train_single_target(end_state, data_size = 100):
    state = end_state.reshape(3,3)
    dict_state = {}
    step = 0
    q = queue.Queue()
    q.put(state)
    cnt = 0
    dict_state[str(state)] = (state, state2dict(state), step)
    while cnt < data_size:
        cur_state = q.get()
        px,py = 0,0
        for i in range(3):
            for j in range(3):
                if cur_state[i][j] == 0:
                    px, py = i, j
                    break
        for k in range(4):
            qx, qy = px + dx[k], py + dy[k]
            if qx >= 0 and qx < 3 and qy >=0 and qy < 3:
                new_state = copy.deepcopy(cur_state)
                new_state[px][py],new_state[qx][qy] = new_state[qx][qy],new_state[px][py]
                if str(new_state) not in dict_state:
                    q.put(new_state)
                    dict_state[str(new_state)]=(new_state, state2dict(new_state), dict_state[str(cur_state)][-1] + 1)
                    cnt += 1
    return dict_state

In [4]:
def generate_train(batch_size=1000, batch_num=1000):
    df = pd.DataFrame()
    state = np.arange(0, 9).reshape(3,3)
    for _i in range(batch_num):
        np.random.shuffle(state)
        dict_state = generate_train_single_target(state, batch_size)
        list_state = [list(dict_state[i][0].reshape(-1)) for i in dict_state]
        list_step = [dict_state[i][-1] for i in dict_state]
        temp_dict = {}
        temp_dict["Step"] = list_step
        new_df = pd.DataFrame()
        new_df["Step"] = list_step
        for i in range(3):
            for j in range(3):
                new_df["start_idx" + str(i*3+j)] = [dict_state[k][1][i*3+j] for k in dict_state]
        for i in range(3):
            for j in range(3):
                new_df["end_idx" + str(i*3+j)] = [state[i][j] for k in dict_state]
        # print(new_df)
        # break
        # new_df = pd.DataFrame({"StartState": list_state, "Step": list_step, "EndState": [list(state.reshape(-1)) for i in dict_state]})
        # print(new_df)
        df = pd.concat([df, new_df], ignore_index=True)
        if _i % 10 == 0:
            print(f"batch{_i}")
    return df

df = generate_train(batch_size=1000, batch_num=200)
df


batch0
batch10
batch20
batch30
batch40
batch50
batch60
batch70
batch80
batch90
batch100
batch110
batch120
batch130
batch140
batch150
batch160
batch170
batch180
batch190


,Step,start_idx0,start_idx1,start_idx2,start_idx3,start_idx4,start_idx5,start_idx6,start_idx7,start_idx8,end_idx0,end_idx1,end_idx2,end_idx3,end_idx4,end_idx5,end_idx6,end_idx7,end_idx8
0,0,6,7,8,3,4,5,0,1,2,6,7,8,3,4,5,0,1,2
1,1,6,7,8,0,4,5,3,1,2,6,7,8,3,4,5,0,1,2
2,1,6,7,8,3,4,5,1,0,2,6,7,8,3,4,5,0,1,2
3,2,0,7,8,6,4,5,3,1,2,6,7,8,3,4,5,0,1,2
4,2,6,7,8,4,0,5,3,1,2,6,7,8,3,4,5,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200264,11,6,4,1,3,7,2,5,0,8,0,1,2,6,7,8,3,4,5
200265,11,7,0,1,4,6,2,3,5,8,0,1,2,6,7,8,3,4,5
200266,11,7,6,1,4,2,0,3,5,8,0,1,2,6,7,8,3,4,5
200267,11,7,6,1,4,5,2,3,0,8,0,1,2,6,7,8,3,4,5


In [5]:
df.to_csv("train.csv", index=False)
print(df.shape)

(200269, 19)


In [6]:
test_set = generate_train(batch_size=100, batch_num=10)
test_set.to_csv("test.csv", index=False)
print(test_set)

batch0
      Step  start_idx0  start_idx1  start_idx2  start_idx3  start_idx4  \
0        0           0           1           2           6           7   
1        1           1           0           2           6           7   
2        1           6           1           2           0           7   
3        2           1           2           0           6           7   
4        2           1           7           2           6           0   
...    ...         ...         ...         ...         ...         ...   
1014     6           4           0           5           3           6   
1015     6           3           0           4           6           1   
1016     6           3           0           5           6           4   
1017     6           3           4           5           6           8   
1018     6           3           4           5           0           6   

      start_idx5  start_idx6  start_idx7  start_idx8  end_idx0  end_idx1  \
0              8           3